# ALZML

The goal of this project is to use publically available datasets construct a classifier to determine the features that confer Alzheimer's Disease. To do so we are combining SNP data with metadata to generate a feature rich dataset. 

First we will identify ROIs from SNP data that are known to be involved in AD. From there, adjacent features will be extracted to serve as a feature profile of the ROI. Once all ROIs have been profiled we will then run a heirarchical clustering algorithm to determine distinguishing features of these ROIs.

From these features we are hoping to construct an HMM that will be able to identify some of the underrepresented variants that may confer AD. 

Data are sourced from:
- gnomad
- IGAP
- TBI Study
- Zou eGWAS
- Mayo eGWAS
- NG00061
- NG00039

IGAP data will be used as the determinant for a ROI. A +/- 5kb region from the location of the SNP will serve as an anchor to gather local features using bedtools in order to retain positional information. 

### Only run bash block if files need to be parsed properly (shouldn't happen)

In [1]:
%%bash
#head -n 10 ng00061/WES_release3AtlasOnly_vep80_most_severe_consequence_per_gene.txt
# These have different numbers of columns of course. Need to fix this maybe? Will only use the coordinates from this
# for now, this is the primary key to link all the other datasets.
#head -n 10 ng00061/WES_release3AtlasOnly_rolling_flat_annotation.txt
#cut -f1-5 WGS_v1_rolling_flat_annotation.txt > WGS_v1_rolling_flat_annotation.pos_only.txt

#head -n 10 ../IGAP_summary_statistics/IGAP_stage_1_2_combined.txt

# I will join all of the chromosomes into one file.
#cat /home/twaddlac/Hackthan_AD/ng00039/pvalue_only/metaanalysis/pvalueonly_METAANALYSIS1_chr*.TBL | perl -pe 's/^(\d+)-(\d+)/$1\t$2/g' > pvalue_only.tsv
#head -n 10 ../pvalue_only.tsv

#I'm not sure what the difference is between these but I am assuming we get the inverse of controls here.
# We will use the controls for a confusion matrix
#ls ../Mayo_eGWAS/
#head ../Mayo_eGWAS/Hap300_CER_All.txt

# Only going to use the coordinates of the annotation files since they have mismatching columns.
##### Only run this once #####
#cut -f1-5 ../NG00061/WGS_v1_rolling_flat_annotation.txt > ../NG00061/WGS_v1_rolling_flat_annotation.pos_only.txt

## only need to run once
#cat <(grep -m1 '^Marker' /home/twaddlac/Hackthan_AD/ng00039/pvalue_only/pvalue/pvalueonly_METAANALYSIS1_chr10.TBL) <(cat /home/twaddlac/Hackthan_AD/ng00039/pvalue_only/pvalue/*TBL | perl -pe 's/(\d+)-(\d+)/$1\t$2/g'| grep -v '^Marker') > /home/twaddlac/Hackthan_AD/ng00039/pvalue_only/pvalue/pvalue.tsv

## IGAP Data

In [3]:
import pandas as pd
igap1 = pd.read_csv('../IGAP_summary_statistics/IGAP_stage_1.txt', sep='\t')
igap12 = pd.read_csv('../IGAP_summary_statistics/IGAP_stage_1_2_combined.txt', sep='\t')
igap1.rename(columns={
    'Chromosome':'chr',
    'Position':'pos'
}, inplace=True)
igap12.rename(columns={
    'Chromosome':'chr',
    'Position':'pos'
}, inplace=True)

## NG00061

In [4]:
anno = pd.read_csv('../NG00061/WGS_v1_rolling_flat_annotation.pos_only.txt', sep='\t', header=0)

In [5]:
conseq = pd.read_csv('../NG00061/WGS_v1_vep80_most_severe_consequence_per_gene.txt', sep='\t', header=0)

## NG00039

In [5]:
pvalue = pd.read_csv('../ng00039/pvalue_only/pvalue/pvalue.tsv', sep='\t', header=0, index_col=False)
# pvalue.columns = ['chr','pos','allele1','allele2','pvalue']

FileNotFoundError: [Errno 2] File b'../ng00039/pvalue_only/pvalue/pvalue.tsv' does not exist: b'../ng00039/pvalue_only/pvalue/pvalue.tsv'

## Mayo_eGWAS

In [7]:
hapCerAd = pd.read_csv('/home/twaddlac/Hackthan_AD/Mayo_eGWAS/Hap300_CER_AD.txt', sep='\t', header=0)
hapTxAd = pd.read_csv('/home/twaddlac/Hackthan_AD/Mayo_eGWAS/Hap300_TX_AD.txt', sep='\t', header=0)
hapmapCerAd = pd.read_csv('/home/twaddlac/Hackthan_AD/Mayo_eGWAS/HapMap2_CER_AD.txt', sep='\t', header=0)
hapmapTxAd = pd.read_csv('/home/twaddlac/Hackthan_AD/Mayo_eGWAS/HapMap2_TX_AD.txt', sep='\t', header=0)

hapCerAd.rename(columns={'CHR':'chr','BP':'pos'}, inplace=True)
hapTxAd.rename(columns={'CHR':'chr','BP':'pos'}, inplace=True)
hapmapCerAd.rename(columns={'CHR':'chr','BP':'pos'}, inplace=True)
hapmapTxAd.rename(columns={'CHR':'chr','BP':'pos'}, inplace=True)

## TBI Study Expression Data
There's a lot more data for this dataset but we can import it later.

In [7]:
ge = pd.read_csv('../TBI_study/gene_expression_matrix_2016-03-03/fpkm_table_normalized.csv', sep=',', header=0)

## gnomad data

In [ ]:
gnomad = pd.read_csv('../IGAP_summary_statistics/gnomad_gwas_intersect.txt', )

# CRAVAT Data
CRAVAT dumps a sqllite3 file that I've read in. The only interesting tables seem to be variants and genes.

The genes that are implicated in AD are seen below:
Chromosome 11       PICAL
Chromosome 2        BIN1
Chromosome 19      CD33
Chromosome 1        CR1
Chromosome 7       EPHA1
Chromosome 6       TREM2
Chromosome 19.     ABCA7
Chromosome 11      SORL
Chromosome 12.    ADAM10
Chromosome     2.     ADAM17
Chromosome 7    AKAP9
Chromosome 4    UNC5C
Chromosome 19    APOE

In [29]:
import sqlite3
conn = sqlite3.connect("../open-cravat-test/snptest1.sqlite")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

variant_annotator = pd.read_sql_query("SELECT * FROM variant_annotator", conn)
variant = pd.read_sql_query("SELECT * FROM variant", conn)
variant_header  = pd.read_sql_query("SELECT * FROM variant_header", conn)
variant_reportsub = pd.read_sql_query("SELECT * FROM variant_reportsub", conn)
gene_annotator = pd.read_sql_query("SELECT * FROM gene_annotator", conn)
gene = pd.read_sql_query("SELECT * FROM gene", conn)
gene_header  = pd.read_sql_query("SELECT * FROM gene_header", conn)
gene_reportsub = pd.read_sql_query("SELECT * FROM gene_reportsub", conn)
sample = pd.read_sql_query("SELECT * FROM sample", conn)
mapping  = pd.read_sql_query("SELECT * FROM mapping", conn)
variant_filtered = pd.read_sql_query("SELECT * FROM variant_filtered", conn)
gene_filtered = pd.read_sql_query("SELECT * FROM gene_filtered", conn)
info = pd.read_sql_query("SELECT * FROM info", conn)


[('variant_annotator',), ('variant',), ('variant_header',), ('variant_reportsub',), ('gene_annotator',), ('gene',), ('gene_header',), ('gene_reportsub',), ('sample_annotator',), ('sample',), ('sample_header',), ('mapping_annotator',), ('mapping',), ('mapping_header',), ('viewersetup',), ('info',), ('variant_filtered',), ('gene_filtered',)]


In [72]:
variant

,base__uid,base__chrom,base__pos,base__ref_base,base__alt_base,base__note,base__coding,base__hugo,base__transcript,base__so,base__achange,base__all_mappings,clinvar__sig,clinvar__disease_refs,clinvar__disease_names,clinvar__rev_stat,clinvar__id,cosmic__cosmic_id,cosmic__variant_count_tissue,cosmic__variant_count,cosmic__transcript,cosmic__protein_change,dbsnp__snp,denovo__PubmedId,denovo__PrimaryPhenotype,denovo__Validation,esp6500__ea_pop_af,esp6500__aa_pop_af,gnomad__af,gnomad__af_afr,gnomad__af_amr,gnomad__af_asj,gnomad__af_eas,gnomad__af_fin,gnomad__af_nfe,gnomad__af_oth,gnomad__af_sas,gtex__gtex_gene,gtex__gtex_tissue,repeat__repeatclass,repeat__repeatfamily,repeat__repeatname,thousandgenomes__af,thousandgenomes__afr_af,thousandgenomes__amr_af,thousandgenomes__eas_af,thousandgenomes__eur_af,thousandgenomes__sas_af,thousandgenomes__chb_af,thousandgenomes__jpt_af,thousandgenomes__chs_af,thousandgenomes__cdx_af,thousandgenomes__khv_af,thousandgenomes__ceu_af,thousandgenomes__tsi_af,thousandgenomes__fin_af,thousandgenomes__gbr_af,thousandgenomes__ibs_af,thousandgenomes__yri_af,thousandgenomes__lwk_af,thousandgenomes__gwd_af,thousandgenomes__msl_af,thousandgenomes__esn_af,thousandgenomes__asw_af,thousandgenomes__acb_af,thousandgenomes__mxl_af,thousandgenomes__pur_af,thousandgenomes__clm_af,thousandgenomes__pel_af,thousandgenomes__gih_af,thousandgenomes__pjl_af,thousandgenomes__beb_af,thousandgenomes__stu_af,thousandgenomes__itu_af,uk10k_cohort__uk10k_twins_ac,uk10k_cohort__uk10k_twins_af,uk10k_cohort__uk10k_alspac_ac,uk10k_cohort__uk10k_alspac_af,uk10k_cohort__uk10k_ac,uk10k_cohort__uk10k_af,vest__transcript,vest__score,vest__pval,vest__all_results,vest__hugo,vista_enhancer__element,vista_enhancer__features,tagsampler__numsample,tagsampler__samples,tagsampler__tags
0,1,chr19,1028150,N,A,None,None,CNN2,ENST00000562958.6,INT,None,"{""CNN2"":[[""Q99439"",null,""INT"",""ENST00000263097...",None,None,None,None,None,None,None,None,None,None,rs75364577,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,SINE,MIR,MIRc,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,None,None,0,,
1,2,chr19,1036445,N,C,None,Y,CNN2,ENST00000562958.6,SYN,S200S,"{""CNN2"":[[""Q99439"",""S179S"",""SYN"",""ENST00000263...",None,None,None,None,None,None,None,None,None,None,rs930231,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,None,None,0,,
2,3,chr19,1037987,N,T,None,None,CNN2,ENST00000562958.6,UT3,None,"{""CNN2"":[[""Q99439"",null,""UT3"",""ENST00000263097...",None,None,None,None,None,None,None,None,None,None,rs3848640,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,None,None,0,,
3,4,chr19,1039187,N,A,None,None,ABCA7,ENST00000263094.10,2KU,None,"{""ABCA7"":[[""Q8IZY2"",null,""2KU"",""ENST0000026309...",None,None,None,None,None,None,None,None,None,None,rs4807468,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,None,None,0,,
4,5,chr19,1039445,N,T,None,None,ABCA7,ENST00000263094.10,2KU,None,"{""ABCA7"":[[""Q8IZY2"",null,""2KU"",""ENST0000026309...",None,None,None,None,None,None,None,None,None,None,rs3795065,None,None,None,None,No

In [64]:
# Genes implicatd in AD
ad_genes = [
    'CD33',
    'ABCA7',
    'APOE'
]

In [70]:
# Only ABCA7 is in our data?!
gene[gene['base__hugo'].isin(ad_genes)]

,base__hugo,base__num_variants,base__so,base__all_so,base__note,biogrid__biogrid,biogrid__acts,biogrid__id,gnomad_gene__transcript,gnomad_gene__oe_lof,gnomad_gene__oe_mis,gnomad_gene__oe_syn,gnomad_gene__lof_z,gnomad_gene__mis_z,gnomad_gene__syn_z,gnomad_gene__pLI,gnomad_gene__pRec,gnomad_gene__pNull,ncbigene__ncbi_desc,ncbigene__entrez,pubmed__n,pubmed__term,uniprot__acc
0,ABCA7,29,MIS,"MIS(4),SYN(3),INT(18),UT5(1),UT3(1),2KU(2)",None,SYNCRIP[26496610]|SGTB[26496610]|SORT1[2649661...,GABRA3;HNRNPD;HTR3A;LLGL2;SGTB;SORT1;SYNCRIP,115629,ENST00000263094;ENST00000433129;ENST0000043568...,0.88848;0.88848;0.95115;0.29017;0.74468;0.20425,1.0813;1.0813;1.0782;1.1342;1.0093;1.0699,1.0306;1.0306;1.032;0.91341;1.0426;0.92329,1.046;1.046;0.44037;1.7368;1.2332;1.641,-1.0897;-1.0897;-1.0152;-0.48163;-0.064653;-0....,-0.5931;-0.5931;-0.60091;0.47053;-0.41313;0.3802,4.4592e-55;4.4592e-55;4.3994e-57;0.2108;2.9369...,7.812e-07;7.812e-07;2.6638e-08;0.75196;0.46651...,1.0;1.0;1.0;0.03724;0.53349;0.038102,The protein encoded by this gene is a member o...,10347,7.0,http://www.ncbi.nlm.nih.gov/pubmed?term=ABCA7[...,Q8IZY2


In [49]:
# variant.merge(gene, left_on='base__hugo', right_on='base__hugo', how='left')
var = variant.merge(gene, on='base__hugo', how='outer')

# Joining Datasets
The file linking everything together will be the annotation data from NG00061 as it should hold the complete set of SNPs. I will use coordinates from this table to join.

In [8]:
headers = ['chr','pos']
temp = anno.merge(pvalue, how='left', left_on=headers, right_on=headers)

In [9]:
temp

,chr,pos,alt_allele,seq_meta_var_id,epacts_var_id,Allele1,Allele2,P-value
0,10,60494,G,"10:60494A,G",10:60494_A/G,a,g,0.32420
1,10,60523,G,"10:60523T,G",10:60523_T/G,t,g,0.37210
2,10,61331,G,"10:61331A,G",10:61331_A/G,a,g,0.75160
3,10,61334,A,"10:61334G,A",10:61334_G/A,NaN,NaN,NaN
4,10,61646,G,"10:61646A,G",10:61646_A/G,NaN,NaN,NaN
5,10,61654,A,"10:61654G,A",10:61654_G/A,NaN,NaN,NaN
6,10,61766,A,"10:61766G,A",10:61766_G/A,NaN,NaN,NaN
7,10,62450,A,"10:62450G,A",10:62450_G/A,NaN,NaN,NaN
8,10,66295,T,"10:66295C,T",10:66295_C/T,NaN,NaN,NaN
9,10,66326,G,"10:66326A,G",10:66326_A/G,a,g,0.58810


In [ ]:
# Breaks kernel. Don't run in notebook.
from functools import reduce
dataframes = [
    anno,
    pvalue,
    conseq,
    igap1,
    igap12,
    hapCerAd,
    hapTxAd,
    hapmapCerAd,
    hapmapTxAd
]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['chr','pos'],
                                                how='outer'), dataframes).fillna('void')